# Stray Dog Management (2025–2027) — Python Analysis

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
plt.rcParams['figure.figsize']=(9,4)
p = Path('.')
cities = pd.read_csv(p/'dim_cities.csv')
cal = pd.read_csv(p/'dim_calendar.csv', parse_dates=['Date'])
ngos = pd.read_csv(p/'dim_ngos.csv')
inc = pd.read_csv(p/'fact_incidents.csv', parse_dates=['Date'])
abc = pd.read_csv(p/'fact_abc.csv', parse_dates=['Date'])
ado = pd.read_csv(p/'fact_adoptions.csv', parse_dates=['Date'])
bud = pd.read_csv(p/'fact_budget.csv')
inc['Year']=inc['Date'].dt.year; inc['Month']=inc['Date'].dt.month
abc['Year']=abc['Date'].dt.year; abc['Month']=abc['Date'].dt.month
ado['Year']=ado['Date'].dt.year; ado['Month']=ado['Date'].dt.month


## KPIs

In [ ]:
kpis = {
 'Total Incidents': len(inc),
 'Resolved %': round(inc['Resolved'].mean()*100,2),
 'Avg Response Time (mins)': round(inc['ResponseTimeMins'].mean(),2),
 'Total ABC Surgeries': int(abc['ABC_Surgeries'].sum()),
 'Total Vaccinations': int(abc['Vaccinations'].sum()),
 'Total Intake': int(ado['ShelterIntake'].sum()),
 'Total Adoptions': int(ado['Adoptions'].sum()),
 'Adoption Rate %': round(ado['Adoptions'].sum()/max(1,ado['ShelterIntake'].sum())*100,2),
 'Budget Utilization %': round(bud['BudgetUtilized'].sum()/bud['BudgetAllocated'].sum()*100,2)
}
kpis

## Trends

In [ ]:
monthly_inc = inc.groupby(['Year','Month'])['IncidentID'].count().reset_index(name='Incidents')
monthly_inc.pivot(index='Month', columns='Year', values='Incidents').plot(title='Monthly Incidents (2025–2027)'); plt.show()

## Export analytics

In [ ]:
city_month = inc.groupby(['State','City','Year','Month']).size().reset_index(name='Incidents')
city_month.to_csv('city_monthly_incidents.csv', index=False)
state_abc = abc.groupby(['State','Year'])[['ABC_Surgeries','Vaccinations']].sum().reset_index()
state_abc.to_csv('state_year_abc.csv', index=False)
adopt_city = ado.groupby(['State','City','Year'])[['ShelterIntake','Adoptions']].sum().reset_index()
adopt_city['AdoptionRate'] = (adopt_city['Adoptions'] / adopt_city['ShelterIntake'].replace(0, pd.NA)).fillna(0)
adopt_city.to_csv('adoption_funnel_city.csv', index=False)
budget_perf = bud.groupby(['State','City','Year','Month'])[['BudgetAllocated','BudgetUtilized']].sum().reset_index()
budget_perf['UtilizationPct'] = (budget_perf['BudgetUtilized']/budget_perf['BudgetAllocated']).fillna(0)
budget_perf.to_csv('budget_performance.csv', index=False)
print('Artifacts saved.')